In [1]:
!pip install tensorflow >> /dev/null
!pip install tensorflow-hub >> /dev/null
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
bert = hub.KerasLayer('./output/bert_wwm_zh_pool/')

In [3]:
bert([['嗷嗷'], ['嗷']]).shape

TensorShape([2, 768])

In [11]:
x = [
    ['我爱你'],
    ['我恨你'],
    ['爱你'],
    ['恨你'],
    ['爱'],
    ['恨'],
]
y = [
    1, 0, 1, 0, 1, 0
]

tx = tf.constant(x)
ty = tf.constant(y)

model = tf.keras.Sequential([
  hub.KerasLayer('./output/bert_wwm_zh_pool/'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', metrics=['accuracy'])
model.fit(tx, ty, epochs=10, batch_size=6)
logits = model.predict(tx)
pred = tf.math.round(logits).numpy().flatten().tolist()

Train on 6 samples
Epoch 1/10
6/6 [==============================] - 7s 1s/sample - loss: 0.8716 - accuracy: 0.5000
Epoch 2/10
6/6 [==============================] - 0s 13ms/sample - loss: 0.7175 - accuracy: 0.5000
Epoch 3/10
6/6 [==============================] - 0s 13ms/sample - loss: 0.6596 - accuracy: 0.5000
Epoch 4/10
6/6 [==============================] - 0s 14ms/sample - loss: 0.6386 - accuracy: 0.6667
Epoch 5/10
6/6 [==============================] - 0s 14ms/sample - loss: 0.6263 - accuracy: 0.8333
Epoch 6/10
6/6 [==============================] - 0s 14ms/sample - loss: 0.6165 - accuracy: 0.8333
Epoch 7/10
6/6 [==============================] - 0s 14ms/sample - loss: 0.6078 - accuracy: 0.8333
Epoch 8/10
6/6 [==============================] - 0s 14ms/sample - loss: 0.5996 - accuracy: 1.0000
Epoch 9/10
6/6 [==============================] - 0s 14ms/sample - loss: 0.5918 - accuracy: 0.8333
Epoch 10/10
6/6 [==============================] - 0s 14ms/sample - loss: 0.5842 - accuracy:

In [12]:
print(pred)
print(y)

[1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
[1, 0, 1, 0, 1, 0]


In [13]:
model.save('/tmp/test_model')

INFO:tensorflow:Assets written to: /tmp/test_model/assets


In [14]:
tf.keras.models.load_model('/tmp/test_model')([['我爱你']])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.54622084]], dtype=float32)>

In [15]:
tf.keras.models.load_model('/tmp/test_model')([['恨你']])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.40016043]], dtype=float32)>